In [8]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from tkinter.font import Font
import psycopg2
from bs4 import BeautifulSoup
import os
from datetime import datetime
from tkinterdnd2 import TkinterDnD, DND_FILES


class DnDFrame(ttk.LabelFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.drop_target_register(DND_FILES)
        self.dnd_bind('<<Drop>>', self.handle_drop)
        
        self.file_listbox = tk.Listbox(self, height=5)
        self.file_listbox.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        btn_frame = ttk.Frame(self)
        btn_frame.pack(fill=tk.X, pady=5)
        
        self.clear_btn = ttk.Button(btn_frame, text="Очистить", command=self.clear_files)
        self.clear_btn.pack(side=tk.LEFT, padx=5)
        
        self.add_btn = ttk.Button(btn_frame, text="Добавить файлы", command=self.add_files)
        self.add_btn.pack(side=tk.RIGHT, padx=5)
    
    def handle_drop(self, event):
        files = self.tk.splitlist(event.data)
        for file in files:
            if file.lower().endswith('.html'):
                if file not in self.file_listbox.get(0, tk.END):
                    self.file_listbox.insert(tk.END, file)
            else:
                messagebox.showwarning("Неверный формат", f"Файл {file} не является HTML")
    
    def add_files(self):
        filetypes = (('HTML files', '*.html'), ('All files', '*.*'))
        files = filedialog.askopenfilenames(title='Выберите файлы', filetypes=filetypes)
        if files:
            for file in files:
                if file not in self.file_listbox.get(0, tk.END):
                    self.file_listbox.insert(tk.END, file)
    
    def clear_files(self):
        self.file_listbox.delete(0, tk.END)
    
    def get_files(self):
        return list(self.file_listbox.get(0, tk.END))

class Application(TkinterDnD.Tk):
    def __init__(self):
        super().__init__()
        self.title("Парсер вакансий HH")
        self.geometry("600x500")
        self.configure(bg='#f0f0f0')
        
        self.DB_CONFIG = {
            'host': 'localhost',
            'port': '5433',
            'database': 'hh_test',
            'user': 'postgres',
            'password': '123'
        }
        
        self.create_widgets()
    
    def create_widgets(self):
        # Стилизация
        bold_font = Font(family="Arial", size=10, weight="bold")
        normal_font = Font(family="Arial", size=10)
        
        # Главный фрейм
        main_frame = ttk.Frame(self, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Заголовок
        ttk.Label(
            main_frame,
            text="Парсер вакансий HeadHunter",
            font=Font(family="Arial", size=12, weight="bold")
        ).pack(pady=(0, 10))
        
        # Область для перетаскивания файлов
        drag_frame = DnDFrame(main_frame, text="Перетащите HTML-файлы сюда или нажмите кнопку", 
                             width=500, height=150)
        drag_frame.pack(fill=tk.BOTH, pady=10, padx=10, expand=True)
        self.drag_frame = drag_frame
        
        # Кнопка обработки
        self.process_btn = ttk.Button(
            main_frame,
            text="Обработать файлы",
            command=self.process_files,
            style='Accent.TButton'
        )
        self.process_btn.pack(pady=10)
        
        # Статус бар
        self.status_var = tk.StringVar()
        status_bar = ttk.Label(
            main_frame,
            textvariable=self.status_var,
            relief=tk.SUNKEN,
            anchor=tk.W
        )
        status_bar.pack(fill=tk.X, pady=(10, 0))
        
        # Стиль
        style = ttk.Style()
        style.configure('Accent.TButton', font=bold_font, foreground='black')
    
    def process_files(self):
        files = self.drag_frame.get_files()
        if not files:
            messagebox.showwarning("Ошибка", "Нет файлов для обработки")
            return
        
        total_records = 0
        processed_files = 0
        
        self.process_btn.config(state=tk.DISABLED)
        self.status_var.set("Начата обработка файлов...")
        self.update()
        
        for file in files:
            try:
                data, metadata = self.parser(file)
                if self.load_db(data, metadata):
                    total_records += len(data)
                    processed_files += 1
                    self.status_var.set(f"Обработан {os.path.basename(file)} ({len(data)} записей)")
                    self.update()
                else:
                    messagebox.showerror("Ошибка", f"Ошибка при сохранении данных из {file}")
            except Exception as e:
                messagebox.showerror("Ошибка", f"Ошибка при обработке {file}: {str(e)}")
        
        self.status_var.set(f"Готово! Обработано {processed_files}/{len(files)} файлов, {total_records} записей")
        self.process_btn.config(state=tk.NORMAL)
    
    def parser(self, pathway):
        data = []
        with open(pathway, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')

        for load_vac in soup.find_all(class_='resume-card-content--pA9euQ2yPckXrBh1'):
            start_time = datetime.now()
            file_name = os.path.basename(pathway)
            file_time_create = datetime.fromtimestamp(os.path.getctime(pathway))
                                   
            main_info = load_vac.find(class_='column-content--q3SfppwQANVUv38P')
            url_find = main_info.find('a')['href']
            info_about_time_to_be = main_info.find(class_='magritte-text___pbpft_4-1-1 magritte-text_style-secondary___1IU11_4-1-1 magritte-text_typography-label-3-regular___Nhtlp_4-1-1')
            
            a = info_about_time_to_be.find(string=lambda t: 'Был' in t)
            if a:
                date = a.find_next('span').text
                full_text = a + ' ' + date
            else:
                full_text = None
            
            data.append((url_find, full_text))
        
        parsing_end = datetime.now()
        metadata = (file_name, file_time_create, start_time, parsing_end)
        return data, metadata
    
    def load_db(self, data, metadata):
        try: 
            with psycopg2.connect(**self.DB_CONFIG) as connect:
                with connect.cursor() as cur:
                    cur.execute('''CREATE TABLE IF NOT EXISTS info_res (
                            id SERIAL PRIMARY KEY,
                            session_id INTEGER,
                            full_url TEXT,
                            visit_time TEXT)''')
                    
                    cur.execute('''CREATE TABLE IF NOT EXISTS parsing_metadata (
                                    id SERIAL PRIMARY KEY,
                                    session_id INTEGER,
                                    html_file_name TEXT,
                                    file_creation_time TIMESTAMP,
                                    parsing_start_time TIMESTAMP,
                                    parsing_end_time TIMESTAMP);''')
                    
                    cur.execute("SELECT COALESCE(MAX(session_id), 0) FROM info_res")
                    
                    max_id = cur.fetchone()[0] + 1
                    data_with_session = [(max_id, url, visit_time) for url, visit_time in data]
                    metadata_with_session = (max_id,) + metadata

                    cur.executemany('INSERT INTO info_res (session_id, full_url, visit_time) VALUES (%s, %s, %s)', data_with_session)
                    cur.execute('INSERT INTO parsing_metadata (session_id, html_file_name, file_creation_time, parsing_start_time, parsing_end_time) VALUES (%s, %s, %s, %s, %s)', metadata_with_session)
                    connect.commit()
                return True
            
        except Exception as e:
            print(f'Ошибка {e}')
            return False

if __name__ == "__main__":
    app = Application()
    app.mainloop()

In [129]:
import pandas as pd

try:
    with psycopg2.connect(**DB_CONFIG) as connect:
        with connect.cursor() as cur:
            cur.execute('SELECT * FROM info_res')
            data = cur.fetchall()  # Получаем все строки
            columns = [desc[0] for desc in cur.description]  # Получаем названия колонок
    df = pd.DataFrame(data, columns=columns)
    print(df.head())
except Exception as e:
    print(f"Ошибка: {e}")

   id                                           full_url    visit_time
0   1  https://hh.ru/resume/bcea4f870002959d7b00ba83c...  Была сегодня
1   2  https://hh.ru/resume/7630fde9000129f42d00ba83c...  Была сегодня
2   3  https://hh.ru/resume/a68b329c00059411bf00ba83c...   Был сегодня
3   4  https://hh.ru/resume/c782f9bf0002db143400ba83c...  Была сегодня
4   5  https://hh.ru/resume/7455f9fb00040b149f00ba83c...   Был сегодня


(Column(name='id', type_code=23),
 Column(name='full_url', type_code=25),
 Column(name='visit_time', type_code=25))

In [147]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import psycopg2
import webbrowser

DB_CONFIG = {
    'host': 'localhost',
    'port': '5433',
    'database': 'hh_test',
    'user': 'postgres',
    'password': '123'
}

# Глобальные переменные
df = pd.DataFrame()  # Основной DataFrame
comments = {}  # Словарь для хранения комментариев: {id_строки: "текст"}
current_edit = None  # Текущая редактируемая ячейка

def load_data():
    """Загружает данные из PostgreSQL в DataFrame"""
    global df
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            df = pd.read_sql('SELECT * FROM info_res', connect)
        # Добавляем пустую колонку для комментариев, если её нет
        if 'comment' not in df.columns:
            df['comment'] = ''
        return df
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось загрузить данные: {e}")
        return pd.DataFrame()

def on_double_click(event):
    """Обработчик двойного клика для редактирования комментариев"""
    global current_edit
    
    # Определяем ячейку, по которой кликнули
    item = table.identify_row(event.y)
    column = table.identify_column(event.x)
    
    # Проверяем, что кликнули по колонке с комментариями
    if table.heading(column)['text'] == 'comment':
        # Получаем текущее значение
        current_value = table.item(item, 'values')[df.columns.get_loc('comment')]
        
        # Создаем поле для редактирования
        x, y, width, height = table.bbox(item, column)
        current_edit = ttk.Entry(table_frame)
        current_edit.place(x=x, y=y, width=width, height=height)
        current_edit.insert(0, current_value)
        current_edit.focus()
        
        # Привязываем событие потери фокуса и нажатия Enter
        current_edit.bind('<FocusOut>', lambda e: save_comment(item, column))
        current_edit.bind('<Return>', lambda e: save_comment(item, column))

def save_comment(item, column):
    """Сохраняет комментарий и удаляет поле ввода"""
    global current_edit
    
    if current_edit:
        new_comment = current_edit.get()
        # Обновляем значение в таблице
        values = list(table.item(item, 'values'))
        values[df.columns.get_loc('comment')] = new_comment
        table.item(item, values=values)
        
        # Сохраняем комментарий в словаре
        comments[item] = new_comment
        
        # Удаляем поле ввода
        current_edit.destroy()
        current_edit = None

def filter_data():
    """Фильтрует данные по всем критериям (включая комментарии)"""
    keyword = keyword_entry.get().strip().lower()
    min_date = min_date_entry.get()
    max_date = max_date_entry.get()
    comment_filter = comment_filter_entry.get().strip().lower()

    filtered_df = df.copy()
    
    # Фильтр по URL
    if keyword:
        filtered_df = filtered_df[filtered_df['full_url'].str.lower().str.contains(keyword, na=False)]
    
    # Фильтр по дате
    if min_date:
        filtered_df = filtered_df[filtered_df['visit_time'] >= min_date]
    if max_date:
        filtered_df = filtered_df[filtered_df['visit_time'] <= max_date]
    
    # Фильтр по комментариям (из словаря comments)
    if comment_filter:
        filtered_items = [
            item for item, comment in comments.items() 
            if comment_filter in comment.lower()
        ]
        filtered_df = filtered_df[filtered_df.index.isin([int(i.lstrip('I')) for i in filtered_items])]

    update_table(filtered_df)

def update_table(dataframe):
    """Обновляет таблицу с комментариями"""
    for row in table.get_children():
        table.delete(row)
    
    for index, row in dataframe.iterrows():
        item_id = table.insert("", "end", values=tuple(row))
        
        # Восстанавливаем комментарии (если есть)
        if item_id in comments:
            table.set(item_id, 'comment', comments[item_id])
        
        # Подсветка ссылки
        table.item(item_id, tags=('clickable',))

def on_link_click(event):
    """Открывает ссылку при клике на full_url"""
    item = table.identify_row(event.y)
    col = table.identify_column(event.x)
    
    # Проверяем, что кликнули по колонке full_url
    if table.heading(col)['text'] == 'full_url':
        url = table.item(item, 'values')[df.columns.get_loc('full_url')]
        if url and isinstance(url, str) and url.startswith(('http://', 'https://')):
            webbrowser.open(url)

# Создаём интерфейс
root = tk.Tk()
root.title("Фильтр резюме HH с комментариями")
root.geometry("1000x700")

# Панель фильтров
filter_frame = ttk.Frame(root, padding="10")
filter_frame.pack(fill=tk.X)

ttk.Label(filter_frame, text="Ключевое слово в URL:").grid(row=0, column=0, padx=5, pady=5)
keyword_entry = ttk.Entry(filter_frame)
keyword_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

ttk.Label(filter_frame, text="Дата от:").grid(row=1, column=0, padx=5, pady=5)
min_date_entry = ttk.Entry(filter_frame)
min_date_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")

ttk.Label(filter_frame, text="Дата до:").grid(row=2, column=0, padx=5, pady=5)
max_date_entry = ttk.Entry(filter_frame)
max_date_entry.grid(row=2, column=1, padx=5, pady=5, sticky="ew")

ttk.Label(filter_frame, text="Фильтр по комментариям:").grid(row=3, column=0, padx=5, pady=5)
comment_filter_entry = ttk.Entry(filter_frame)
comment_filter_entry.grid(row=3, column=1, padx=5, pady=5, sticky="ew")

filter_button = ttk.Button(filter_frame, text="Применить фильтры", command=filter_data)
filter_button.grid(row=4, column=0, columnspan=2, pady=10)

# Таблица
table_frame = ttk.Frame(root)
table_frame.pack(fill=tk.BOTH, expand=True)

# Загружаем данные и добавляем колонку 'comment'
df = load_data()
if 'comment' not in df.columns:
    df['comment'] = ''

columns = list(df.columns)
table = ttk.Treeview(table_frame, columns=columns, show="headings")

# Настраиваем колонки
for col in columns:
    table.heading(col, text=col)
    table.column(col, width=120)

# Подсветка ссылок
table.tag_configure('clickable', foreground='blue', font=('Arial', 10, 'underline'))

# Полоса прокрутки
scrollbar = ttk.Scrollbar(table_frame, orient="vertical", command=table.yview)
table.configure(yscrollcommand=scrollbar.set)
scrollbar.pack(side="right", fill="y")
table.pack(fill=tk.BOTH, expand=True)

# Привязка событий
table.bind('<Double-1>', on_double_click)  # Двойной клик для редактирования
table.bind('<Button-1>', on_link_click)    # Одинарный клик для открытия ссылки

# Заполняем таблицу
update_table(df)

root.mainloop()

C:\Users\sekin\AppData\Local\Temp\ipykernel_17004\515635631.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM info_res', connect)


In [194]:
import os
from datetime import datetime

datetime.fromtimestamp(os.path.getctime(r'C:\Users\sekin\VisualStudioProjects\learn_pars\тест_2.html'))

datetime.datetime(2025, 8, 15, 13, 50, 54, 459742)